In [0]:
dbutils.fs.mounts()

In [0]:
dbutils.fs.unmount('/mnt/sales')

In [0]:
dbutils.secrets.listScopes()

In [0]:
dbutils.secrets.list('capstone-dl')

In [0]:

client_id = dbutils.secrets.get(scope='capstone-dl', key='client-id')
secret_value = dbutils.secrets.get(scope='capstone-dl', key='secretvalue')
tenant_id = dbutils.secrets.get(scope='capstone-dl', key='tenantid')

configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": client_id,
  "fs.azure.account.oauth2.client.secret": secret_value,
  "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/" + tenant_id + "/oauth2/token"
}

dbutils.fs.mount(
  source = "abfss://sales@capstoneadls.dfs.core.windows.net/",
  mount_point = "/mnt/sales",
  extra_configs = configs
)



In [0]:
dbutils.fs.ls('/mnt/sales/landing')

In [0]:
# Define connection parameters
dbServer = 'capstoneweek'  # already includes .database.windows.net in the URL
dbPort = '1433'
dbName = 'capstoneproject'
dbUser = 'capstone'
databricksScope = 'capstone-dl'

# Retrieve password securely from Databricks secret scope
dbPassword = dbutils.secrets.get(scope=databricksScope, key='sql-passwd')

# Construct JDBC URL
connectionUrl = f'jdbc:sqlserver://{dbServer}.database.windows.net:{dbPort};database={dbName};user={dbUser}'

# Define connection properties
connectionProperties = {
    'password': dbPassword,
    'driver': 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}

# Read data from SQL table into Spark DataFrame
validStatusDf = spark.read.jdbc(
    url=connectionUrl,
    table='dbo.valid_order_status',
    properties=connectionProperties
)

# Display the DataFrame
display(validStatusDf)


In [0]:
ordersDf = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/sales/landing/week24_orders.csv")

In [0]:
ordersDf.groupBy('order_id').count().filter('count > 1').show()

In [0]:

invalidDf = ordersDf.filter(~col("order_status").isin(valid_status))


In [0]:
synapse_server_name = "capstone-ws.sql.azuresynapse.net"
synapse_db_name = "capstonededicated"
synapse_user_name = "sqladminuser"
synapse_password = "Gokul!23$"

synapse_jdbc_url = f"jdbc:sqlserver://{synapse_server_name}:1433;database={synapse_db_name};user={synapse_user_name};password={synapse_password};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.sql.azuresynapse.net;loginTimeout=30;"


In [0]:
ordersDf.write \
  .format("jdbc") \
  .option("url", synapse_jdbc_url) \
  .option("dbtable", "dbo.orders") \
  .option("user", "sqladminuser") \
  .option("password", "Gokul!23$") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .mode("append") \
  .save()